In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from tensorflow.keras.applications import EfficientNetB4
# from efficientnet.keras import EfficientNetB3

from tensorflow.keras.optimizers import Adam

# ignoring warnings
import warnings
warnings.simplefilter("ignore")

import os, cv2, json
from PIL import Image

In [2]:
def create_model():

    conv_base = EfficientNetB4(include_top = False, weights = None,
                               input_shape = (380, 380, 3))
    model = conv_base.output
    model = layers.Flatten()(model)
    model = layers.Dense(5, activation = "softmax")(model)
    model = models.Model(conv_base.input, model)

    model.compile(optimizer = Adam(lr = 0.0002),
                  loss = "sparse_categorical_crossentropy",
                  metrics = ["acc"])
    return model

In [3]:
model = create_model()
model.summary()

________________
block6d_add (Add)               (None, 12, 12, 272)  0           block6d_drop[0][0]               
                                                                 block6c_add[0][0]                
__________________________________________________________________________________________________
block6e_expand_conv (Conv2D)    (None, 12, 12, 1632) 443904      block6d_add[0][0]                
__________________________________________________________________________________________________
block6e_expand_bn (BatchNormali (None, 12, 12, 1632) 6528        block6e_expand_conv[0][0]        
__________________________________________________________________________________________________
block6e_expand_activation (Acti (None, 12, 12, 1632) 0           block6e_expand_bn[0][0]          
__________________________________________________________________________________________________
block6e_dwconv (DepthwiseConv2D (None, 12, 12, 1632) 40800       block6e_expand_activation[0

In [4]:
model.load_weights('Models/Varadh;s model weights/SavedWeights.hdf5')

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\harsh\AppData\Local\Temp\tmpt6hkkhb9\assets


In [8]:
tflite_interpreter = tf.lite.Interpreter(model_path="model.tflite")

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])
print("type:", output_details[0]['dtype'])

== Input details ==
shape: [  1 380 380   3]
type: <class 'numpy.float32'>

== Output details ==
shape: [1 5]
type: <class 'numpy.float32'>


In [9]:
tflite_interpreter.resize_tensor_input(input_details[0]['index'], (32, 380   , 380   , 3))
tflite_interpreter.resize_tensor_input(output_details[0]['index'], (32, 5))
tflite_interpreter.allocate_tensors()

input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

print("== Input details ==")
print("shape:", input_details[0]['shape'])
print("\n== Output details ==")
print("shape:", output_details[0]['shape'])

== Input details ==
shape: [ 32 380 380   3]

== Output details ==
shape: [32  5]


In [ ]:
# Set batch of images into input tensor
tflite_interpreter.set_tensor(input_details[0]['index'], val_image_batch)
# Run inference
tflite_interpreter.invoke()
# Get prediction results
tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)

# >> Prediction results shape: (32, 5)

# Convert prediction results to Pandas dataframe, for better visualization
tflite_pred_dataframe = pd.DataFrame(tflite_model_predictions)
tflite_pred_dataframe.columns = dataset_labels

In [11]:
train_labels = pd.read_csv("train.csv")
train_labels.head()

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3


In [17]:
BATCH_SIZE = 32
STEPS_PER_EPOCH = len(train_labels)*0.8 / BATCH_SIZE
VALIDATION_STEPS = len(train_labels)*0.2 / BATCH_SIZE
EPOCHS = 25
TARGET_SIZE = 380

In [18]:
train_labels.label = train_labels.label.astype('str')

train_datagen = ImageDataGenerator(validation_split = 0.20,
                                     rotation_range = 30,
                                     zoom_range = 0.15,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     brightness_range=[0.4,1.4],
                                     fill_mode = 'nearest')
                                    #  height_shift_range = 0.1,
                                    #  width_shift_range = 0.1

train_generator = train_datagen.flow_from_dataframe(train_labels,
                         directory ="train_images",
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")


validation_datagen = ImageDataGenerator(validation_split = 0.20)

validation_generator = validation_datagen.flow_from_dataframe(train_labels,
                         directory = "train_images",
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (TARGET_SIZE, TARGET_SIZE),
                         batch_size = BATCH_SIZE,
                         class_mode = "sparse")

Found 17118 validated image filenames belonging to 5 classes.
Found 4279 validated image filenames belonging to 5 classes.


In [19]:
val_image_batch, val_label_batch = next(iter(validation_generator))

In [21]:
# Set batch of images into input tensor
tflite_interpreter.set_tensor(input_details[0]['index'], val_image_batch)
# Run inference
tflite_interpreter.invoke()
# Get prediction results
tflite_model_predictions = tflite_interpreter.get_tensor(output_details[0]['index'])
print("Prediction results shape:", tflite_model_predictions.shape)

# >> Prediction results shape: (32, 5)

# Convert prediction results to Pandas dataframe, for better visualization
tflite_pred_dataframe = pd.DataFrame(tflite_model_predictions)
tflite_pred_dataframe.columns = ["CBB","CBSD", "CGM", "CMD", "Healthy"]

Prediction results shape: (32, 5)


In [22]:
tflite_pred_dataframe.head()

,CBB,CBSD,CGM,CMD,Healthy
0,0.021328,0.011257,0.011563,0.018089,0.937763
1,0.006764,0.021918,0.242174,0.359220,0.369924
2,0.000965,0.230051,0.008106,0.758398,0.002480
3,0.001718,0.000655,0.011324,0.971596,0.014708
4,0.029705,0.025812,0.028361,0.239592,0.676530
